In [1]:
!pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-8w2988wf
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-8w2988wf
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 40.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567358 sha256=807f878c7956da85a6eecc81a4f7bb76ef898f8d2372438e87b948420ac593f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-h8

In [2]:
!pip install transformers

In [3]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 535.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existin

In [4]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dateutil-2.8.2:
      Successfully uninstalled python-dateutil-2.8.2


In [5]:
!pip install torch

In [7]:
import gradio as gr
from transformers import pipeline
from mistralai import Mistral
import numpy as np
import torch
from bark import generate_audio

# Load Whisper model for transcription
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

# Function to transcribe audio using Whisper
def transcribe(audio):
    try:
        sr, y = audio
        y = y.astype(np.float32)
        y /= np.max(np.abs(y))

        transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]
        return transcription
    except Exception as e:
        return f"Error during transcription: {str(e)}"

# Function to fetch response from Mistral AI
def mistral_search(transcribed_text, api_key):
    try:
        client = Mistral(api_key=api_key)
        messages = [{"role": "user", "content": transcribed_text}]
        response = client.chat.complete(model="mistral-large-latest", messages=messages)
        ai_response = response.choices[0].message.content
        return ai_response
    except Exception as e:
        return f"Error during AI response: {str(e)}"

# Function to convert text to audio
def text_to_speech(ai_response):
    try:
        audio_data = generate_audio(ai_response)
        return audio_data
    except Exception as e:
        return f"Error during TTS generation: {str(e)}"

# Gradio interface setup
def combined_function(audio, api_key):
    try:
        transcribed_text = transcribe(audio)

        # Update the Gradio interface with the transcribed text
        gr.update(value=transcribed_text, label="Transcribed Text")

        # Search in MistralAI asynchronously
        ai_response_future = mistral_search.submit(transcribed_text, api_key)

        # Wait for the MistralAI response and update the interface
        ai_response = ai_response_future.result()
        if "Error" in ai_response:
            return transcribed_text, ai_response, None
        gr.update(value=ai_response, label="AI Response")

        # Convert the AI response to audio asynchronously
        audio_data_future = text_to_speech.submit(ai_response)

        # Wait for the audio generation and update the interface
        audio_data = audio_data_future.result()
        if "Error" in audio_data:
            return transcribed_text, ai_response, None
        gr.update(value=(audio_data, 44100), label="Response Audio")

        return transcribed_text, ai_response, (audio_data, 44100)
    except Exception as e:
        return f"Error in combined function: {str(e)}"

# Create the Gradio interface
demo = gr.Interface(
    fn=combined_function,
    inputs=[
        gr.Audio(sources="microphone", type="numpy", label="Record Your Question"),
        gr.Textbox(label="OpenAI API Key", type="password")
    ],
    outputs=[
        gr.Textbox(label="Transcribed Text"),
        gr.Textbox(label="AI Response"),
        gr.Audio(label="Response Audio")
    ],
    title="AI Voice Assistant with Mistral and Bark",
    description="Transcribe audio, search in MistralAI, and convert the response to speech using Whisper and Bark."
)

# Launch the interface
demo.launch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://94ef23d9fbbee4154d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [8]:
import gradio as gr
from transformers import pipeline
from mistralai import Mistral
import numpy as np
import torch
from bark import generate_audio  # Assuming you're using Suno's Bark TTS

# Load Whisper model for transcription
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

# Function to transcribe audio using Whisper
def transcribe(audio):
    try:
        sr, y = audio
        y = y.astype(np.float32)
        y /= np.max(np.abs(y))

        # Transcribe the audio
        transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]
        return transcription
    except Exception as e:
        return f"Error during transcription: {str(e)}"

# Function to fetch response from Mistral AI and convert it to audio
def mistral_search_and_tts(transcribed_text, api_key):
    try:
        client = Mistral(api_key=api_key)

        # Search in MistralAI using the transcribed text
        messages = [{"role": "user", "content": transcribed_text}]
        response = client.chat.complete(model="mistral-large-latest", messages=messages)
        ai_response = response.choices[0].message.content

        # Convert text response to audio using Suno Bark TTS
        audio_data = generate_audio(ai_response)

        # Return both the AI response text and the generated audio
        return ai_response, audio_data
    except Exception as e:
        return f"Error during AI response or TTS generation: {str(e)}", None

# Gradio interface setup
def combined_function(audio, api_key):
    transcribed_text = transcribe(audio)

    # Check if transcription was successful
    if "Error" in transcribed_text:
        return transcribed_text, "No AI response due to transcription error", None

    ai_response, audio_data = mistral_search_and_tts(transcribed_text, api_key)

    # Check if AI response or TTS generation was successful
    if "Error" in ai_response:
        return transcribed_text, ai_response, None

    return transcribed_text, ai_response, (audio_data, 44100)  # Assuming a 44.1 kHz sample rate for Bark TTS

# Create the Gradio interface
demo = gr.Interface(
    fn=combined_function,
    inputs=[
        gr.Audio(sources="microphone", type="numpy", label="Record Your Question"),
        gr.Textbox(label="OpenAI API Key", type="password")
    ],
    outputs=[
        gr.Textbox(label="Transcribed Text"),
        gr.Textbox(label="AI Response"),
        gr.Audio(label="Response Audio")
    ],
    title="AI Voice Assistant with Mistral and Bark",
    description="Transcribe audio, search in MistralAI, and convert the response to speech using Whisper and Bark."
)

# Launch the interface
demo.launch()


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://10be75786f3d2e9a1f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
